# Coursera Capstone Final Project 
## The Battle of Neighborhoods (Week 1)
### Mariano Puga - Part 1 : Introduction and Data Sections

### We import the libraries

In [1]:
import pandas as pd
import numpy as np
import json

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install geopy
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Coursera Capstone - REPORT CONTENT
1. Introduction Section : ⁃ Discussion of the business problem and the interested audience in this project.
2. Data Section:
⁃ Description of the data that will be used to solve the problem and the sources.
3. Methodology section: 
⁃ Discussion and description of exploratory data analysis carried out, any inferential statistical testing performed, and if any machine learnings were used establishing the strategy and purposes.
4. Results section: 
⁃ Discussion of the results.
5. Discussion section: 
⁃ Elaboration and discussion on any observations noted and any recommendations suggested based on the results.
6. Conclusion section: 
⁃ Report Conclusion.

### A description of the problem and a discussion of the background. (15 marks)


# 1. Introduction Section :

## Discussion of the business problem and the audience who would be interested in this project.
### Description of the Problem and Background
**Scenario:**

I'm currently working in Santiago (Chile, a small country in south america for those who don't know), but I wanna work overseas to get a hold of deeper experiences and develop new abilities. Also, I'm a huge surfing fan so I'd love to combine a city big enough to study an MBA and also stay close to the ocean.   
The first place I thought of in order to do this was Melbourne, Australia. I want new experiences so I wont compare my current situation (venues I tend to go to for example) to my new prospects but I'd love to find a place with the following amenities:

- Residential Apartment in order to share it (hopefully 3 bedrooms)
- Easy access to the surf spots.
- Desirable to have venues such as coffee shops, Asian restaurants and gym's
- Rent shouldn't be too high, I found most of my information on https://www.kaggle.com/anthonypino/melbourne-housing-market and there wasn't information on rent prices so I'll use housing prices to get an idea of how expensive is each suburb.  

#### Business Problem:
The challenge is to find a suitable suburb to live in in Melbourne that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.

#### Interested Audience
I'm particulary interested in surfing so I guess this is oriented to those recent graduates that want to explore studying/working close to the ocean 

### A description of the data and how it will be used to solve the problem. (15 marks)


# 2. Data Section:

## Description of the data and its sources that will be used to solve the problem

### Description of the Data:

The following data is required to answer the issues of the problem:

1. List of suburbs of Melbourne with their geodata (latitud and longitud).
2. List of housing prices for each suburb.
3. Venues for each of Melbourne's neighborhoods.


## How the data will be used to solve the problem
### The data will be used as follows:

- Use Foursquare and geopy data to map top 10 venues for all Melbourne neighborhoods and clustered in groups 
- Evaluate housing in Melbourne for each suburb.
- Addresses from posible locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
- Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies.

(https://www.kaggle.com/anthonypino/melbourne-housing-market)
(https://data.melbourne.vic.gov.au/)


### The procesing of these DATA will allow to answer the key questions to make a decision:

- Which suburb is more expensive?
- What is the area of Melbourne with best rental pricing that meets criteria established? (estimated based on house prices)
- What is the distance from the ocean and the tentative future home?
- How do the prices compare?
- Can I have a really good coffee just before I go surfing?
- Are there tradeoffs between size and price and location?
- Any other interesting statistical data findings of the real estate and overall data.


### First we upload de diferent suburbs of Melbourne (we use this table only to check if we have all our suburbs in df2)

In [18]:
url1 = 'https://raw.githubusercontent.com/marianopuga/Battle-of-the-neighborhoods/master/Melbourne.xls'
read_table = pd.read_csv(url1,header=[0])
df1 = read_table
df1

,City,Neighborhood
0,Melbourne,Carlton
1,Melbourne,Carlton North
2,Melbourne,Docklands
3,Melbourne,East Melbourne
4,Melbourne,Flemington
5,Melbourne,Kensington
6,Melbourne,Melbourne 3000
7,Melbourne,Melbourne 3004
8,Melbourne,North Melbourne
9,Melbourne,Port Melbourne


### Then we upload the coordinates for each suburb

In [19]:
url2 = 'https://raw.githubusercontent.com/marianopuga/Battle-of-the-neighborhoods/master/Address_Points%20(1).csv'
read_table = pd.read_csv(url2,header=[0])
df2 = read_table
df2

,StreetNo,GISID,the_geom,AddComp,StrName,Suburb,AddressPnt,SuburbID,StreetID,EASTING,NORTHING,LATITUDE,LONGITUDE
0,84,28064,POINT (144.979502658185 -37.835383325166),NaN,Hope Street,South Yarra,84 Hope Street South Yarra,595.0,720,322200.715825,5.810526e+06,-37.835383,144.979503
1,261,26268,POINT (144.984359089201 -37.848932002343),NaN,Punt Road,Melbourne,261 Punt Road Melbourne,591.0,1004,322660.576327,5.809032e+06,-37.848932,144.984359
2,129,57067,POINT (144.966043133987 -37.814669852174),City Pillar Opposite,Swanston Street,Melbourne,City Pillar Opposite 129 Swanston Street Melb...,591.0,1114,320966.124201,5.812799e+06,-37.814670,144.966043
3,1,20179,POINT (144.96875067939 -37.810206322362),NaN,Jones Lane,Melbourne,1 Jones Lane Melbourne,591.0,748,321193.704159,5.813299e+06,-37.810206,144.968751
4,26,9563,POINT (144.977804675755 -37.817028046999),NaN,Wellington Crescent,East Melbourne,26 Wellington Crescent East Melbourne,587.0,1168,322007.177009,5.812559e+06,-37.817028,144.977805
5,29,54207,POINT (144.961846725915 -37.823294521978),NaN,Power Street,Southbank,29 Power Street Southbank,596.0,1346,320617.591002,5.811834e+06,-37.823295,144.961847
6,495,10692,POINT (144.974179390899 -37.809519854086),NaN,Albert Street,East Melbourne,495 Albert Street East Melbourne,587.0,375,321669.979611,5.813386e+06,-37.809520,144.974179
7,20,35522,POINT (144.945620594716 -37.794806537703),NaN,Erskine Street,North Melbourne,20 Erskine Street North Melbourne,592.0,611,319119.825032,5.814963e+06,-37.794807,144.945621
8,54,28401,POINT (144.984642530147 -37.842417064547),NaN,Pasley Street,South Yarra,54 Pasley Street South Yarra,595.0,965,322669.908267,5.809755e+06,-37.842417,144.984643
9,9,62712,POINT (144.942256563844 -37.805990926179),NaN,Stawell Street,West Melbourne,9 Stawell Street West Melbourne,597.0,117835,318850.934294,5.813716e+06,-37.805991,144.942257


### Drop unnecesary Data

In [20]:
df2.drop(['GISID','the_geom','AddComp','EASTING','NORTHING','AddressPnt','StrName','StreetNo','StreetID'], axis = 1, inplace = True)
df2

,Suburb,SuburbID,LATITUDE,LONGITUDE
0,South Yarra,595.0,-37.835383,144.979503
1,Melbourne,591.0,-37.848932,144.984359
2,Melbourne,591.0,-37.814670,144.966043
3,Melbourne,591.0,-37.810206,144.968751
4,East Melbourne,587.0,-37.817028,144.977805
5,Southbank,596.0,-37.823295,144.961847
6,East Melbourne,587.0,-37.809520,144.974179
7,North Melbourne,592.0,-37.794807,144.945621
8,South Yarra,595.0,-37.842417,144.984643
9,West Melbourne,597.0,-37.805991,144.942257


### Now we group 

In [75]:
Melbourne = df2.groupby('Suburb').mean().reset_index()
Melbourne

,Suburb,SuburbID,LATITUDE,LONGITUDE
0,Carlton,585.0,-37.799815,144.967612
1,Carlton North,586.0,-37.783667,144.960043
2,Docklands,599.0,-37.818229,144.944143
3,East Melbourne,587.0,-37.813617,144.983546
4,Flemington,588.0,-37.787583,144.918486
5,Kensington,590.0,-37.794071,144.926097
6,Melbourne,591.0,-37.814637,144.963924
7,North Melbourne,592.0,-37.799290,144.946400
8,Parkville,593.0,-37.787375,144.952417
9,Port Melbourne,594.0,-37.827094,144.916919


In df1 we have:

##### 6	Melbourne	Melbourne 3000
##### 7	Melbourne	Melbourne 3004
    
We simply asume Melbourne as the combination of both for simplicity.

In [52]:
address = 'Melbourne'
geolocator = Nominatim()
location = geolocator.geocode(address)
Latitude = location.latitude
Longitude = location.longitude
print('The geograpical coordinate of Melbourne are {},{}.'.format(Latitude,Longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of Melbourne are -37.8142176,144.9631608.


In [76]:
# We create a map of Melbourne using latitude and longitude values using Folium
map_Melbourne = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Melbourne['LATITUDE'], Melbourne['LONGITUDE'], Melbourne['Suburb']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Melbourne)  
    
map_Melbourne

# NOW THAT WE HAVE THE CITY SEGMENTED BY SUBURB WITH THE RESPECTIVE COORDINATES, I'D LIKE TO KNOW HOUSING PRICES FOR A 3 BEDROOM APARTMENT


In [55]:
url3 = 'https://raw.githubusercontent.com/marianopuga/Battle-of-the-neighborhoods/master/Melbourne_housing_FULL.csv'
read_table = pd.read_csv(url3,header=[0])
df3 = read_table
df3

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.80140,144.99580,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.79960,144.99840,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.80790,144.99340,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.81140,145.01160,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.80930,144.99440,Northern Metropolitan,4019.0
5,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra City Council,-37.79690,144.99690,Northern Metropolitan,4019.0
6,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra City Council,-37.80720,144.99410,Northern Metropolitan,4019.0
7,Abbotsford,16 Maugie St,4,h,NaN,SN,Nelson,6/08/2016,2.5,3067.0,...,2.0,2.0,400.0,220.0,2006.0,Yarra City Council,-37.79650,144.99650,Northern Metropolitan,4019.0
8,Abbotsford,53 Turner St,2,h,NaN,S,Biggin,6/08/2016,2.5,3067.0,...,1.0,2.0,201.0,NaN,1900.0,Yarra City Council,-37.79950,144.99740,Northern Metropolitan,4019.0
9,Abbotsford,99 Turner St,2,h,NaN,S,Collins,6/08/2016,2.5,3067.0,...,2.0,1.0,202.0,NaN,1900.0,Yarra City Council,-37.79960,144.99890,Northern Metropolitan,4019.0


## DROP UNNECESSARY DATA

In [56]:
df3.drop(['Method','SellerG','Distance','Postcode','Car','BuildingArea','YearBuilt','CouncilArea','Regionname','Propertycount'], axis = 1, inplace = True)
df3

,Suburb,Address,Rooms,Type,Price,Date,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude
0,Abbotsford,68 Studley St,2,h,NaN,3/09/2016,2.0,1.0,126.0,-37.80140,144.99580
1,Abbotsford,85 Turner St,2,h,1480000.0,3/12/2016,2.0,1.0,202.0,-37.79960,144.99840
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,4/02/2016,2.0,1.0,156.0,-37.80790,144.99340
3,Abbotsford,18/659 Victoria St,3,u,NaN,4/02/2016,3.0,2.0,0.0,-37.81140,145.01160
4,Abbotsford,5 Charles St,3,h,1465000.0,4/03/2017,3.0,2.0,134.0,-37.80930,144.99440
5,Abbotsford,40 Federation La,3,h,850000.0,4/03/2017,3.0,2.0,94.0,-37.79690,144.99690
6,Abbotsford,55a Park St,4,h,1600000.0,4/06/2016,3.0,1.0,120.0,-37.80720,144.99410
7,Abbotsford,16 Maugie St,4,h,NaN,6/08/2016,3.0,2.0,400.0,-37.79650,144.99650
8,Abbotsford,53 Turner St,2,h,NaN,6/08/2016,4.0,1.0,201.0,-37.79950,144.99740
9,Abbotsford,99 Turner St,2,h,NaN,6/08/2016,3.0,2.0,202.0,-37.79960,144.99890


In [57]:
df3.drop(['Address','Date','Type'], axis = 1, inplace = True)

In [58]:
df3

,Suburb,Rooms,Price,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude
0,Abbotsford,2,NaN,2.0,1.0,126.0,-37.80140,144.99580
1,Abbotsford,2,1480000.0,2.0,1.0,202.0,-37.79960,144.99840
2,Abbotsford,2,1035000.0,2.0,1.0,156.0,-37.80790,144.99340
3,Abbotsford,3,NaN,3.0,2.0,0.0,-37.81140,145.01160
4,Abbotsford,3,1465000.0,3.0,2.0,134.0,-37.80930,144.99440
5,Abbotsford,3,850000.0,3.0,2.0,94.0,-37.79690,144.99690
6,Abbotsford,4,1600000.0,3.0,1.0,120.0,-37.80720,144.99410
7,Abbotsford,4,NaN,3.0,2.0,400.0,-37.79650,144.99650
8,Abbotsford,2,NaN,4.0,1.0,201.0,-37.79950,144.99740
9,Abbotsford,2,NaN,3.0,2.0,202.0,-37.79960,144.99890


In [62]:
Melbourne_houseprices = df3[df3['Bedroom2'] == 3.0].reset_index(drop=True)
Melbourne_houseprices.head()

,Suburb,Rooms,Price,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude
0,Abbotsford,3,NaN,3.0,2.0,0.0,-37.8114,145.0116
1,Abbotsford,3,1465000.0,3.0,2.0,134.0,-37.8093,144.9944
2,Abbotsford,3,850000.0,3.0,2.0,94.0,-37.7969,144.9969
3,Abbotsford,4,1600000.0,3.0,1.0,120.0,-37.8072,144.9941
4,Abbotsford,4,NaN,3.0,2.0,400.0,-37.7965,144.9965


## WE GROUP BY SUBURB AND GROUP ONLY ACCOMMODATIONS WITH 3 BEDROOMS

In [77]:
Melbourne_houseprices2 = Melbourne_houseprices.groupby('Suburb').mean().reset_index()
Melbourne_houseprices2

,Suburb,Rooms,Price,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude
0,Abbotsford,3.000000,1.239323e+06,3.0,1.571429,282.916667,-37.803435,144.997004
1,Aberfeldie,3.080000,1.377773e+06,3.0,1.800000,516.958333,-37.759272,144.895449
2,Airport West,3.040541,8.158768e+05,3.0,1.445946,477.071429,-37.723299,144.879728
3,Albanvale,3.000000,5.324167e+05,3.0,1.454545,569.000000,-37.745064,144.768961
4,Albert Park,3.034483,2.065359e+06,3.0,1.586207,183.423077,-37.844668,144.953288
5,Albion,2.969697,6.733667e+05,3.0,1.303030,513.468750,-37.779966,144.817644
6,Alphington,3.050000,1.461062e+06,3.0,1.500000,533.388889,-37.778238,145.027809
7,Altona,3.017241,8.943352e+05,3.0,1.448276,487.905660,-37.865453,144.820215
8,Altona Meadows,3.000000,6.364091e+05,3.0,1.588235,465.200000,-37.880479,144.780984
9,Altona North,3.016667,8.111509e+05,3.0,1.200000,537.418182,-37.834557,144.848733


In [81]:
Melbourne_houseprices3 = Melbourne_houseprices2.loc[(Melbourne_houseprices2['Suburb'] == 'Carlton')|
                                                   (Melbourne_houseprices2['Suburb'] == 'Carlton North')|
                                                   (Melbourne_houseprices2['Suburb'] == 'East Melbourne')|
                                                   (Melbourne_houseprices2['Suburb'] == 'Flemington')|
                                                   (Melbourne_houseprices2['Suburb'] == 'Kensington')|
                                                   (Melbourne_houseprices2['Suburb'] == 'Melbourne')|
                                                   (Melbourne_houseprices2['Suburb'] == 'North Melbourne')|
                                                   (Melbourne_houseprices2['Suburb'] == 'Parkville')|
                                                   (Melbourne_houseprices2['Suburb'] == 'Port Melbourne')|
                                                   (Melbourne_houseprices2['Suburb'] == 'South Wharf')|
                                                   (Melbourne_houseprices2['Suburb'] == 'South Yarra')|
                                                   (Melbourne_houseprices2['Suburb'] == 'Southbank')|
                                                   (Melbourne_houseprices2['Suburb'] == 'West Melbourne')].reset_index(drop=True) 

                                                   
Melbourne_houseprices3                                                   
                                         

,Suburb,Rooms,Price,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude
0,Carlton,3.000000,1.238938e+06,3.0,1.428571,898.894737,-37.799670,144.964539
1,Carlton North,2.970588,1.661591e+06,3.0,1.294118,220.060606,-37.784755,144.973818
2,East Melbourne,3.000000,2.362500e+06,3.0,2.000000,77.500000,-37.815092,144.985514
3,Flemington,3.000000,1.099500e+06,3.0,1.241379,238.923077,-37.784436,144.927914
4,Kensington,3.020833,1.092933e+06,3.0,1.312500,369.883721,-37.793306,144.926958
5,Melbourne,3.000000,1.012000e+06,3.0,2.166667,15.833333,-37.810040,144.966160
6,North Melbourne,3.076923,1.348921e+06,3.0,1.769231,217.619048,-37.799070,144.947444
7,Parkville,3.000000,1.508909e+06,3.0,1.846154,947.454545,-37.785196,144.956013
8,Port Melbourne,3.009804,1.628356e+06,3.0,1.696078,369.285714,-37.836906,144.938268
9,South Yarra,3.000000,1.714935e+06,3.0,1.797980,325.988235,-37.841587,144.992771


# NOW WE SORT OUR HOUSING PRICES IN DECENDING ORDER

In [83]:
Melbourne_houseprices4 = Melbourne_houseprices3.sort_values(by=['Price'], ascending=False).reset_index(drop=True) 
Melbourne_houseprices4

,Suburb,Rooms,Price,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude
0,East Melbourne,3.000000,2.362500e+06,3.0,2.000000,77.500000,-37.815092,144.985514
1,South Yarra,3.000000,1.714935e+06,3.0,1.797980,325.988235,-37.841587,144.992771
2,Carlton North,2.970588,1.661591e+06,3.0,1.294118,220.060606,-37.784755,144.973818
3,Port Melbourne,3.009804,1.628356e+06,3.0,1.696078,369.285714,-37.836906,144.938268
4,Parkville,3.000000,1.508909e+06,3.0,1.846154,947.454545,-37.785196,144.956013
5,North Melbourne,3.076923,1.348921e+06,3.0,1.769231,217.619048,-37.799070,144.947444
6,West Melbourne,3.000000,1.319632e+06,3.0,1.958333,272.263158,-37.808377,144.947529
7,Carlton,3.000000,1.238938e+06,3.0,1.428571,898.894737,-37.799670,144.964539
8,Flemington,3.000000,1.099500e+06,3.0,1.241379,238.923077,-37.784436,144.927914
9,Kensington,3.020833,1.092933e+06,3.0,1.312500,369.883721,-37.793306,144.926958


# With this we end our data exploration
.........................................................................................................................................




# Coursera Capstone Project



## The Battle of Neighborhoods (Week 2)



### Mariano Puga - Part 2 : METHODOLOGY, RESULTS, DISCUSSION AND CONCLUSION
































# 3. Methodology section

### NOW WE DEFINE THE FORESQUARE CREDENTIALS AND VERSION WE ARE USING

In [44]:
CLIENT_ID = '0GGVWFVY53QQXPAFMF5EBLBMGTEV01Q1Q121HZESIB2ETE1H' # your Foursquare ID
CLIENT_SECRET = 'TGGYTLBAT13J2PRP15P1E0MBAKMETADT2OYUU3KO2C4U5ZNL' # your Foursquare Secret
VERSION = '20200202' # Foursquare API version

### BACK TO MELBOURNE, WE GET THE TOP 100 VENUES WITHIN A 500 MTS RADIUS

In [45]:
LIMIT = 100
radius = 500

In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url4 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url4).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category' ]
    
    return(nearby_venues)

In [47]:
Melbourne_venues = getNearbyVenues(names=Melbourne['Suburb'],
                                   latitudes=Melbourne['LATITUDE'],
                                   longitudes=Melbourne['LONGITUDE']
                                    )

South Yarra


KeyError: 'groups'

In [ ]:
print(Melbourne_venues.shape)
Melbourne_venues.head()

In [ ]:
print('There are {} uniques categories.'.format(len(Melbourne_venues['Venue Category'].unique())))

#### WE ANALYZE THE SUBURBS, WE CREATE DUMMIES IN ORDER TO CHECK WHICH TYPE OF VENUES WE HAVE

In [ ]:
# one hot encoding
Melbourne_onehot = pd.get_dummies(Melbourne_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Melbourne_onehot['Suburb'] = Melbourne_venues['Suburb'] 

# move neighborhood column to the first column
fixed_columns = [Melbourne_onehot.columns[-1]] + list(Melbourne_onehot.columns[:-1])
Melbourne_onehot = Melbourne_onehot[fixed_columns]

Melbourne_onehot.head()

In [ ]:
#RESULTS

In [ ]:
#DISCUSSION

In [ ]:
#CONCLUSION